In [2]:
# install necessary libraries (assuming the initial block has been run)
# !pip install scikit-learn
# !pip install pandas
# !pip install nltk

# download necessary nltk resources (assuming the initial block has been run)
# import nltk
# nltk.download('punkt')
# nltk.download('punkt_tab')
# nltk.download('wordnet')
# nltk.download('stopwords')

print("NLTK Resources downloaded successfully!")

# Import necessary libraries
import pandas as pd
import numpy as np
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# 🌟 替换：导入 MLPClassifier (Scikit-learn 中的 ANN 实现)
from sklearn.neural_network import MLPClassifier 
from sklearn.metrics import classification_report, accuracy_score
from joblib import dump,load

# Import NLTK
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re
# 导入标签编码器，因为 ANN 模型通常需要数字标签
from sklearn.preprocessing import LabelEncoder

# Initialize NLTK resources
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

df = pd.read_csv('dataset.csv')

# shuffle the data for robust splitting
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
print(df.head())

def preprocess_text(text):
    # 1. Convert to Lowercase
    text = text.lower()
    
    # 2. Remove Punctuation and Special Characters
    text = re.sub(r'[^\w\s]', '', text)
    
    # 3. Tokenization
    tokens = word_tokenize(text)
    
    # 4. Stopword Removal
    tokens = [word for word in tokens if word not in stop_words]
    
    # 5. Lemmatization (Key Enhancement)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Rejoin tokens into a single string
    return ' '.join(tokens)

# Apply the new preprocessing function to the text column
df['cleaned_text'] = df['text'].apply(preprocess_text)
print("--- Preprocessing Complete (with NLTK Lemmatization) ---")
print(df[['text', 'cleaned_text']].head())

# Prepare the cleaned text and intents for the model training section
X = df['cleaned_text']
y = df['intent']

# Initialize the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the text data to create the feature matrix X
X = vectorizer.fit_transform(X)
# y = df['intent'] # y 已经指向 df['intent']，这行是冗余的

# 🌟 新增：对标签进行数字编码 (ANN/MLP 的常见要求)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
print("--- Intent labels have been encoded to numbers ---")

print(f"Feature matrix X shape: {X.shape}")
print(f"Labels y (encoded) shape: {y_encoded.shape}")


X_train, X_test, y_train, y_test = train_test_split(
    X,y_encoded, # 使用编码后的标签进行训练和测试
    test_size=0.2,
    random_state=42,
    stratify=y_encoded if len(df['intent'].unique()) > 1 else None,
)

print(f"Train set size:{X_train.shape[0]} samples")
print(f"Test set size:{X_test.shape[0]} samples")

# 🌟 替换：实例化 ANN (MLPClassifier) 模型
# hidden_layer_sizes=(100, 50): 两个隐藏层，分别有 100 和 50 个神经元
# activation='relu': 激活函数 (常用的选择)
# max_iter=300: 最大的迭代次数
# solver='adam': 优化器
# alpha=0.0001: L2 正则化参数
# verbose=True: 打印训练过程
ann_model = MLPClassifier(
    hidden_layer_sizes=(100, 50),
    activation='relu',
    max_iter=300,
    solver='adam',
    alpha=0.0001,
    random_state=42
)

# 训练模型
# ⚠️ 注意: 稀疏矩阵 (如 TfidfVector 结果) 可能会导致警告或性能下降。
# MLPClassifier 通常更喜欢密集矩阵。
# 如果数据量小，可以执行 X_train.toarray()
ann_model.fit(X_train, y_train)

# Make predictions on the test set
pred_encoded = ann_model.predict(X_test)

# 🌟 新增：将预测结果解码回原始标签名称
pred = le.inverse_transform(pred_encoded)
y_test_decoded = le.inverse_transform(y_test)

print("--- ANN (MLPClassifier) Model Evaluation ---")
print("Accuracy:", accuracy_score(y_test_decoded, pred))
print("\nClassification Report:\n")
if len(df['intent'].unique()) > 1:
    print(classification_report(y_test_decoded, pred, zero_division=0))
else:
    print("Classification Report skipped: Only one class in test set.")

# 🌟 替换：保存训练好的 ANN 模型、Vectorizer 和 LabelEncoder
dump(ann_model, 'ann_intent_model.joblib')
dump(vectorizer, 'tfidf_vectorizerANN.joblib')
dump(le, 'label_encoder.joblib') # 保存 LabelEncoder 至关重要
print("Model, Vectorizer, and LabelEncoder saved using joblib.")

# Predefined fixed responses (Retrieval System)
responses = {
    "ask_room_price": "Our rooms start from RM180 per night.",
    "ask_availability": "We currently have several rooms available.",
    "ask_facilities": "We offer free Wi-Fi, breakfast, pool, gym and parking.",
    "ask_location": "We are located in Kuala Lumpur City Centre (KLCC).",
    "ask_checkin_time" : "Check-in time is from 2:00 PM.",
    "ask_checkout_time" : "Check-out time is at 12:00 PM.",
    "ask_booking" : "You can book directly through our website or at the front desk.",
    "ask_cancellation" : "Cancellations are free up to 24 hours before arrival.",
    "greeting" : "Hello! How may I assist you today?",
    "goodbye" : "Goodbye! Have a great day!"
}

# 🌟 替换：更新函数名，并接收 LabelEncoder
def chatbot_reply_ann(user_input, model, vectorizer, label_encoder, responses):
    # 1. Preprocessing
    user_input_cleaned = preprocess_text(user_input)
    
    # 2. Feature Extraction: Transform the input using the fitted vectorizer
    vector = vectorizer.transform([user_input_cleaned])

    # 3. Intent Prediction (返回编码的数字)
    intent_encoded = model.predict(vector)[0]

    # 4. 🌟 新增：将预测结果解码回原始标签名称
    intent = label_encoder.inverse_transform([intent_encoded])[0]

    # 5. Retrieval (Check for unknown intent/fallback)
    return responses.get(intent, f"Sorry, I predicted the intent '{intent}', but I don't have a specific response for that yet. Please rephrase your question.")

# Test the chatbot function
print("\n --- ANN (MLPClassifier) Chatbot Test ---")
test_input = "Do you have parking?"
predicted_response = chatbot_reply_ann(test_input, ann_model, vectorizer, le, responses)
print(f"User Input: {test_input}")
print(f"Chatbot Reply: {predicted_response}")

NLTK Resources downloaded successfully!
                                text             intent
0                       Good evening           greeting
1           When does check-out end?  ask_checkout_time
2  What is your cancellation policy?   ask_cancellation
3      What is your check-in policy?   ask_checkin_time
4          When does check-in start?   ask_checkin_time
--- Preprocessing Complete (with NLTK Lemmatization) ---
                                text         cleaned_text
0                       Good evening         good evening
1           When does check-out end?         checkout end
2  What is your cancellation policy?  cancellation policy
3      What is your check-in policy?       checkin policy
4          When does check-in start?        checkin start
--- Intent labels have been encoded to numbers ---
Feature matrix X shape: (100, 105)
Labels y (encoded) shape: (100,)
Train set size:80 samples
Test set size:20 samples
--- ANN (MLPClassifier) Model Evaluation ---
Accu